# B-tree Implementation for Key-value Store
#### An example project illustring B-tree implementation in a key-value store, with numeric or non-numeric unique keys, range look-up, and functional and performance tests

In [1]:
# To structure code automatically
%load_ext nb_black

<IPython.core.display.Javascript object>

### Import B-tree Implementation

In [2]:
from btree import BTree

<IPython.core.display.Javascript object>

### KVStore Implementation

In [3]:
class KVStore(BTree):
    def __init__(self, min_possible_key, max_possible_key, split_threshold=2):
        super().__init__(split_threshold)
        self.min_possible_key = str(min_possible_key)
        self.max_possible_key = str(max_possible_key)

    def _add(self, current_node, key, value):
        key = str(key)
        if current_node.is_leaf():
            len_before = len(current_node)
            current_node.insert_entry(key, value)
            if len(current_node) > len_before:
                self.size += 1
        else:
            child_index = current_node.get_insert_index(key)
            self._add(current_node.children[child_index], key, value)
        if len(current_node) > self.split_threshold:
            parent = current_node.split()
            if current_node == self.root:
                self.root = parent
                self.height += 1

    def add(self, key, value):
        self._add(self.root, key, value)

    def _range_query(
        self, current_node, range_start, range_end, min_key, max_key,
    ):
        if range_end < min_key or range_start > max_key:
            return []
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key <= range_end:
                results.append(current_node.values[i])
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node) else max_key
                results += self._range_query(
                    child, range_start, range_end, new_min_key, new_max_key
                )
        return results

    def range_query(self, range_start, range_end):
        return self._range_query(
            self.root,
            str(range_start),
            str(range_end),
            str(self.min_possible_key),
            str(self.max_possible_key),
        )

<IPython.core.display.Javascript object>